In [ ]:
# !pip install google-cloud-dialogflow

In [ ]:
# !pip install --user python-telegram-bot --upgrade

In [ ]:
# !pip install pymorphy2[fast]

In [1]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

Чат-бот с Dialgflow

In [2]:
from google.cloud import dialogflow

In [3]:
updater = Updater("5014531389:AAG--uPXfGCjBWbuT_bjtW7a01fJmgSYuTg", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = './data/t-bot-slja-79ec87fe1f5b.json'# скачнный JSON


DIALOGFLOW_PROJECT_ID = 't-bot-slja' #PROJECT ID из DialogFlow 
DIALOGFLOW_LANGUAGE_CODE = 'ru' # язык
SESSION_ID = 'tony_nlp_bot'  # ID бота из телеграма

In [ ]:
# def startCommand(update: Update, context: CallbackContext):
#     update.message.reply_text('Добрый день!')

    
# def textMessage(update: Update, context: CallbackContext):
#     session_client = dialogflow.SessionsClient()
#     session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)

#     text_input = dialogflow.TextInput(text=update.message.text,
#                                             language_code=DIALOGFLOW_LANGUAGE_CODE)
#     query_input = dialogflow.QueryInput(text=text_input)
    
#     try:
#         response = session_client.detect_intent(session=session, query_input=query_input)
#     except InvalidArgument:
#          raise

#     text = response.query_result.fulfillment_text
#     if text:
#         update.message.reply_text(response.query_result.fulfillment_text)
#     else:
#         update.message.reply_text('Что?')

In [ ]:
# on different commands - answer in Telegram
# dispatcher.add_handler(CommandHandler("start", startCommand))
# dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

In [ ]:
# Start the Bot
# updater.start_polling()
# updater.idle()

Чат-бот c Bert

In [4]:
import string
import nltk
import annoy
import numpy as np
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from transformers import TFAutoModel, AutoTokenizer
import pickle

In [30]:
def get_response(question, index, index_map):
    question = preprocess_txt(question)
    tok = tokenizer(question, return_token_type_ids=False, return_tensors='tf')
    vector = bert(**tok)[1].numpy()[0]
    answers, distances = index.get_nns_by_vector(vector, 10, include_distances=True)
    if max(distances) > 0.1:
#         answer_idx = np.argmax(distances)
        answer_idx = distances.index(np.random.choice(distances[-3:]))
        return f'BERT: {index_map[answers[answer_idx]]}'
    else:
        return

In [31]:
def DFtextMessage(text):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(DIALOGFLOW_PROJECT_ID, SESSION_ID)

    text_input = dialogflow.TextInput(text=text,language_code=DIALOGFLOW_LANGUAGE_CODE)
    query_input = dialogflow.QueryInput(text=text_input)
    
    try:
        response = session_client.detect_intent(session=session, query_input=query_input)
    except InvalidArgument:
         raise

    text = response.query_result.fulfillment_text
    if text:
        return f'DF: {text}'
    else:
        return 'END: Что?'

In [32]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return ' '.join(spls)

In [33]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru") + nltk.corpus.stopwords.words('russian'))
exclude = set(string.punctuation)

In [9]:
bert = TFAutoModel.from_pretrained("setu4993/LaBSE")
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at setu4993/LaBSE.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Файлы доступны по ссылкам:  
index_map: https://drive.google.com/file/d/1jBANV0ParwfVAjRnQ6tZ40OmRrgVqTfj/view?usp=sharing  
bert_index: https://drive.google.com/file/d/1s_b5a7LyDi2XtwXKHFYov9zPMURnaW4u/view?usp=sharing

In [10]:
bert_index = annoy.AnnoyIndex(768 , 'angular')
bert_index.load('./data/bert_index')

True

In [11]:
with open('./data/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)

In [34]:
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Hi!')

def text_message(update: Update, context: CallbackContext):
    text = update.message.text
    response = get_response(text, bert_index, index_map)
    if response:
        update.message.reply_text(response)
    else:
        response = DFtextMessage(text)
        update.message.reply_text(response)

In [36]:
# updater = Updater("5014531389:AAG--uPXfGCjBWbuT_bjtW7a01fJmgSYuTg", use_context=True)
# dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, text_message))

# Start the Bot
updater.start_polling()
updater.idle()